In [3]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: openai in c:\users\h p\appdata\local\programs\python\python311\lib\site-packages (from -r requirements.txt (line 1)) (0.28.0)




[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import openai
import os
import json
import pycountry
import base64
import re
from fuzzywuzzy import process
from dotenv import load_dotenv

load_dotenv()
openai.api_key=os.getenv("OPENAI_API_KEY")

with open("visa_information.json","r") as file:
    visa_data =json.load(file)


In [5]:
def get_countries_code(origin_country, destination_country):
    try:
        origin_code = pycountry.countries.lookup(origin_country).alpha_3
        destination_code = pycountry.countries.lookup(destination_country).alpha_3
        return origin_code, destination_code
    
    except LookupError:  # here if an error accord we use fuzzywuzzy if there like typo errors
        
        valid_country_names = [country.name for country in pycountry.countries]
        corrected_origin = process.extractOne(origin_country, valid_country_names)
        corrected_destination = process.extractOne(destination_country, valid_country_names)
        
        if corrected_origin and corrected_destination:
            origin_code = pycountry.countries.lookup(corrected_origin[0]).alpha_3
            destination_code = pycountry.countries.lookup(corrected_destination[0]).alpha_3
            return origin_code, destination_code
        else:
            print("Error: Could not find valid country names.")
            return None, None


In [6]:
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


In [7]:
def detect_issuing_country(image_path):
    encoded_image = encode_image_to_base64(image_path)
    image_message = {
        "role": "user",
        "content": [
            {"type": "text", "text": "Return only the name of the country in this passport image."},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
        ]
    }
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[image_message], 
            max_tokens=50
        )
        detected_country = response.choices[0].message.content.strip()
        return {"detected_country": detected_country}
    except Exception as e:
        return {"error": str(e)}

In [8]:
def get_visa_requirement(origin_country, destination_country):
    origin_code, destination_code = get_countries_code(origin_country, destination_country)
    
    if origin_code is None or destination_code is None or origin_code not in visa_data:
        return {"visaRequirement": "visaRequired",
        "destination": destination_country}
    
    passport_info = visa_data[origin_code]
    
    if "visaFree" in passport_info and destination_code in passport_info["visaFree"]:
        max_stay = passport_info["visaFree"][destination_code]["maxStay"]
        return {
            "visaRequirement": "visaFree",
            "destination": destination_country,
            "maxStay": max_stay
        }
    
    if "visaOnArrival" in passport_info and destination_code in passport_info["visaOnArrival"]:
        max_stay = passport_info["visaOnArrival"][destination_code]["maxStay"]
        return {
            "visaRequirement": "visaOnArrival",
            "destination": destination_country,
            "maxStay": max_stay
        }
    
    return {
        "visaRequirement": "visaRequired",
        "destination": destination_country
    }


In [9]:
def process_visa_function_call(message):
    try:
        args = json.loads(message["function_call"]["arguments"])
        origin = args["origin_country"]
        destination = args["destination_country"]
    
        function_result = get_visa_requirement(origin, destination)
        
        if "error" in function_result:
            response = function_result["error"]
        else:
            visa_req = function_result["visaRequirement"]
            if visa_req == "visaFree":
                response = (f"Thank you for the information. For your journey to {destination}, "
                            f"{origin} passport holders can travel without a visa for {function_result['maxStay']} days.")
            elif visa_req == "visaOnArrival":
                response = (f"Thank you for the information. For your journey to {destination}, "
                            f"{origin} passport holders can obtain a visa upon arrival which will give you a "
                            f"{function_result['maxStay']}-day maximum stay.")
            elif visa_req == "visaRequired":
                response = (f"Thank you for the information. For your journey to {destination}, "
                            f"{origin} passport holders will need to apply for a visa in advance at a "
                            f"{destination} consulate - there are no visa on arrival or visa exemptions.")
        return response

    except Exception as e:
        return ("I'm sorry, there was an error processing your visa requirement information. "
                "Could you please try again?")

In [10]:
functions = [
       {
        "name": "detect_issuing_country",
        "description": "Extracts issuing country from passport image. after it you should get_visa_requirement",
        "parameters": {
            "type": "object",
            "properties": {
                "image_path": {"type": "string", "description": "Path to passport image with double qoutes"}
            },
            "required": ["image_path"]
        }
    },
    {
        "name": "get_visa_requirement",
        "description": "Determines visa requirements between two countries. It accepts the full names of the traveler's passport country and the destination country. Internally, the function will convert these names to ISO alpha-3 country codes and return structured visa information including visa type, maximum stay, and a human-readable message.",
        "parameters": {
            "type": "object",
            "properties": {
                "origin_country": {
                    "type": "string",
                    "description": "Traveler's passport country (full or common name)."
                },
                "destination_country": {
                    "type": "string",
                    "description": "Destination country (full or common name)."
                }
            },
            "required": ["origin_country", "destination_country"]
        }
    }
]

In [11]:
Prompt = """
You are a polite, professional visa assistant whose goal is to help users determine the visa requirements for their travel based on their passport nationality and destination country. Follow these clear, step-by-step instructions for each query:

1. Greet and Gather Information:
   - Begin by greeting the user warmly.
   - If the user does not mention both the passport country and the destination country, ask for the missing detail.
   - **If the user provides an image path (a string enclosed in double quotes) that likely points to a passport image, immediately call the function "detect_issuing_country" with the provided image path. Once you get the issuing country, then call get visa requirements **
   - if you detect an image path, first call detect_issuing_country to get the passport's issuing country, then use that output along with the provided destination country to call get_visa_requirement and return the visa details.

2. Extract Country Names:
   - Clearly extract the origin country (passport issuer) and the destination country from the conversation.
   - Use the full/common country names (e.g., "Singapore", "Cyprus"). Do not convert them to ISO alpha‑3 codes; the backend function will handle that.
   - Ensure both country names are explicitly mentioned before proceeding.
   - - **After calling detect_issuing_country**, immediately use the detected country as the origin and call get_visa_requirement.

3. Determine Visa Requirements:
   - Once both country names are available, call the provided visa-checking function "get_visa_requirement".
   - Verify that the passport country exists in the visa data before returning any details.
   - Identify whether the passport qualifies for visa‑free travel, visa on arrival, or requires a pre‑approved visa.
   - Do not provide any visa details directly; always retrieve them using the function call.

4. Use Function Return Values:
   - When the visa-checking function is called, use the returned values (such as "visaRequirement" and "maxStay") to build your final answer.
   - Format your answer exactly as follows (replace placeholders exactly):
       - Begin with: "Thank you for the information. For your journey to [destination],"
       - Then state:
           - If a visa is required: "[origin country] passport holders will need to apply for a visa in advance at a [destination] consulate – there are no visa on arrival or visa exemptions."
           - If visa on arrival is available: "[origin country] passport holders can obtain a visa upon arrival which will give you a [maxStay]-day maximum stay."
           - If visa-free: "[origin country] passport holders can travel without a visa for [maxStay] days."

5. Respond Clearly and Concisely:
   - Keep responses clear, concise, and limited to 2–3 sentences.

6. Maintain a Professional Multi-Turn Conversation:
   - Keep your tone friendly, professional, and consistent.
   - Ask clarifying questions if any required information is missing or ambiguous.
   - If new or additional details are provided mid-conversation (e.g., a new destination), refer back to the original query and ask for clarification as needed.

7. Handle Errors and Ambiguities:
   - If the visa-checking function returns an error or the passport country is not found in the visa data, politely inform the user and ask if they would like to try with different information.
   - If the user's input deviates from the expected pattern, ask for clarification or additional details.

8. Example Interaction:
     - User: "Hello, I'm planning a trip to Portugal. Can you help me with the visa requirements?"
     - Assistant: "Of course! Could you please specify your passport issuer so I can assist you better with your journey?"
     - User: "My passport is from Cyprus."  
       (Alternatively, the user might provide an image path: "C:\images\passport.jpg")
     - If an image path is provided, immediately call "detect_issuing_country" with the path and simulate a message: "My passport is from Cyprus."
     - Response: "Thank you for the information. For your journey to Portugal, Cypriot passport holders can obtain a visa upon arrival which will give you a 90-day maximum stay."

Always ensure your tone remains friendly and professional, and never provide visa requirement details without first calling the appropriate function.

"""

In [12]:
class ConversationState:
    def __init__(self):
        self.messages = [{"role": "system", "content": Prompt}]
    
    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

In [13]:
def chat_with_ai(user_input, state):
    state.add_message("user", user_input)
    
    # Loop until we get a final content message
    while True:
        response_obj = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=state.messages,
            functions=functions,
            function_call="auto"
        )
        message = response_obj["choices"][0]["message"]
        
        if message.get("function_call"):
            function_name = message["function_call"]["name"]
            arguments = json.loads(message["function_call"]["arguments"])
            
            if function_name == "detect_issuing_country":
                result = detect_issuing_country(arguments["image_path"])
                function_message = {
                "role": "function",
                "name": function_name,
                "content": json.dumps(result)
                  }
                state.messages.append(function_message)
            elif function_name == "get_visa_requirement":
               return process_visa_function_call(message)
            else:
                state.add_message("assistant", "Unknown function call.")
                return "Unknown function call."
        else:
            final_response = message.get("content", "Could you please provide the missing details?")
            state.add_message("assistant", final_response)
            return final_response


In [14]:
if __name__ == "__main__":
    state = ConversationState()
    print("Visa Assistant: Hello! How can I help you today? (Type 'exit' to quit) Also we support images in the format of double quotes path double quotes ")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Visa Assistant: Goodbye!")
            break
        answer = chat_with_ai(user_input, state)
        print("Visa Assistant:", answer)

Visa Assistant: Hello! How can I help you today? (Type 'exit' to quit) Also we support images in the format of double quotes path double quotes 
Visa Assistant: Hello! Thank you for reaching out. To assist you with the visa requirements for your journey to Portugal, could you please confirm the passport country that you hold?
Visa Assistant: Thank you for the information! For your journey to Portugal, Cypriot passport holders can travel without a visa for 90 days.
Visa Assistant: Goodbye!
